In [ ]:
pip install "unsloth[colab-new]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
pip install "transformers" "peft" "accelerate" "bitsandbytes" "datasets"

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name="unsloth/Qwen3-30B-A3B-bnb-4bit"

model, tokenizer= FastLanguageModel.from_pretrained(model_name=model_name, max_seq_length=2048, dtype=None, load_in_4bit=True)

==((====))==  Unsloth 2025.8.9: Fast Qwen3_Moe patching. Transformers: 4.55.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
# Configure LoRA parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
from datasets import load_dataset

# Define your tokenizer and EOS_TOKEN before this
# tokenizer = ...
EOS_TOKEN = tokenizer.eos_token

# NEW formatting function
def formatting_prompts_func(examples):
    contexts = examples['Context']
    responses = examples['Response']
    texts = []
    for context, response in zip(contexts, responses):
        messages = [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ]
        formatted_text = tokenizer.apply_chat_template(
            messages, tokenize = False, add_generation_prompt = False
        )
        texts.append(formatted_text + EOS_TOKEN)
    return { "text" : texts }

# Load and map the dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Your dataset is now correctly formatted and ready for training.
print("Formatting successful!")

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

Formatting successful!


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60, # Set to a specific number of steps for a quick test run
        num_train_epochs = 3, # Or train for a number of epochs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

# Start training!
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/3512 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,512 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 843,841,536 of 31,375,964,160 (2.69% trained)
/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.186800
2,3.237100
3,2.781100
4,2.968000
5,3.235200
6,2.846600
7,2.622100
8,2.777700
9,2.580800
10,2.576800


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=60, training_loss=2.371933428446452, metrics={'train_runtime': 6389.2091, 'train_samples_per_second': 0.075, 'train_steps_per_second': 0.009, 'total_flos': 3.4353691656290304e+16, 'train_loss': 2.371933428446452})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token has not been saved to git credential helper.


In [ ]:
# Push just the LoRA adapters (most common)
model.push_to_hub("Abhishek4545/qwen3-30b-mental-health-buddy-lora")
tokenizer.push_to_hub("Abhishek4545/qwen3-30b-mental-health-buddy-lora")

# Or, if you merged the model first:
# merged_model.push_to_hub("your_hf_username/qwen3-8b-mental-health-buddy-merged")
# tokenizer.push_to_hub("your_hf_username/qwen3-8b-mental-health-buddy-merged")

Saved model to https://huggingface.co/Abhishek4545/qwen3-30b-mental-health-buddy-lora
